In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import datapackage
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import statsmodels.api as sm

## investigating volatility linkages between oil, gas, and regionalised coal futures markets ##

## load data from Github into new frame and remove rows with absent data ##

url = 'https://raw.githubusercontent.com/joe-ascroft/phd/master/data/df-combined.csv'
df = pd.read_csv(url)
df["DATE"] = pd.to_datetime(df["DATE"],dayfirst=True)
for col in df.columns[1:]:
    df[col] = pd.to_numeric(df[col],errors='coerce')

print(df)

           DATE    NCF    RCF    WTI    HH  Brent  NBP_UK  GBP-USD       NBP  \
0    2009-01-01    NaN    NaN    NaN   NaN    NaN     NaN      NaN       NaN   
1    2009-01-02  74.35  81.00  46.17  5.41  46.91   54.74   1.4453  7.911572   
2    2009-01-03    NaN    NaN    NaN   NaN    NaN     NaN      NaN       NaN   
3    2009-01-04    NaN    NaN    NaN   NaN    NaN     NaN      NaN       NaN   
4    2009-01-05  77.65  84.40  48.61  5.83  49.62   56.66   1.4708  8.333553   
...         ...    ...    ...    ...   ...    ...     ...      ...       ...   
4378 2020-12-27    NaN    NaN    NaN   NaN    NaN     NaN      NaN       NaN   
4379 2020-12-28  85.35  71.65    NaN   NaN  50.86   51.07   1.3458  6.873001   
4380 2020-12-29  84.20  70.00    NaN   NaN  51.09   55.51   1.3508  7.498291   
4381 2020-12-30  81.60  69.40    NaN   NaN  51.63   55.88   1.3618  7.609738   
4382 2020-12-31  80.50  69.00    NaN   NaN  51.80   56.40   1.3659  7.703676   

      Unnamed: 9  Unnamed: 10  
0      

In [ ]:
## price return calculations ##

for col in df.columns[1:9]:
    df[col+'_r'] = (df[col].pct_change())
df = df.dropna()
print(df)

In [ ]:
## markov regime switching for univariate analysis ##

mod = sm.tsa.MarkovRegression(df["HH"], k_regimes=2, switching_variance = True)
res_mod = mod.fit()
res_mod.summary()
print(res_mod.summary().as_latex())

In [ ]:
res_mod.filtered_marginal_probabilities[1].plot(
    title='Probability of being in the high regime', figsize=(12,2))

In [ ]:
plt.plot(df["NBP"])

In [ ]:
## markov regime switching for univariate returns analysis ##

mod = sm.tsa.MarkovRegression(df["GBP-USD"], k_regimes=3, switching_variance = True)
res_mod = mod.fit()
res_mod.summary()